# Import and Initialize

In [38]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
from fuzzywuzzy import fuzz

file_path = '/Users/peterwilliams/Downloads/PlaneCrashes/PlaneCrashes.csv'

df = pd.read_csv(file_path)
df = pd.DataFrame(df)

# Cleaning

In [41]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df.columns = pd.Series(df.columns).apply(lambda col: col.lower())
df.dropna(how='all', inplace= True)
df.fillna("unknown", inplace = True)
df.columns = df.columns.str.replace(' ', '_')
df['crew_on_board'] = df['crew_on_board'].replace('unknown', '0.0')
df['crew_on_board'] = pd.to_numeric(df['crew_on_board'], errors='coerce')

df['pax_on_board'] = df['pax_on_board'].replace('unknown', '0.0')
df['pax_on_board'] = pd.to_numeric(df['pax_on_board'], errors='coerce')

df['crew_fatalities'] = df['crew_fatalities'].replace('unknown', '0.0')
df['crew_fatalities'] = pd.to_numeric(df['crew_fatalities'], errors='coerce')

df['pax_fatalities'] = df['pax_fatalities'].replace('unknown', '0.0')
df['pax_fatalities'] = pd.to_numeric(df['pax_fatalities'], errors='coerce')

df['other_fatalities'] = df['other_fatalities'].replace('unknown', '0.0')
df['other_fatalities'] = pd.to_numeric(df['other_fatalities'], errors='coerce')

df['total_on_board'] = df['crew_on_board'] + df['pax_on_board']
df['total_fatalities'] = df['crew_fatalities'] + df['pax_fatalities'] + df['other_fatalities']

df_cleaned = df.drop(['crew_on_board', 'crew_fatalities', 'pax_on_board', 'pax_fatalities', 'other_fatalities'], axis=1)

df.drop(['msn',
         'yom',
         'flight_no.', 
        ], axis=1, inplace=True)

order = [
    "date", "time", "aircraft", "operator", "flight_phase",
    "flight_type", "crash_site", "schedule", "crash_location",
    "country", "region", "circumstances", "crash_cause",
    "total_on_board", "total_fatalities", "survivors"
]

df_cleaned = df_cleaned[order]

In [43]:
passenger_types = [
    "scheduled revenue flight",
    "charter/taxi (non scheduled revenue flight)",
    "private",
    "executive/corporate/business",
    "ferry"
    "cargo",
    "positioning",
    "ambulance",
    "topographic",
    "geographical / geophysical / scientific",
    "illegal (smuggling)",
    "spraying (agricultural)",
    "humanitarian",
    "meteorological / weather",
    "fire fighting"
]

df_cleaned['flight_type'] = df_cleaned['flight_type'].apply(lambda x: 'passenger flights' if x in passenger_types else None)

df_cleaned = df_cleaned.dropna(subset=['flight_type']) 

# Fatal Routes

In [52]:
df_routes = df_cleaned.sort_values(by='schedule', ascending=False)

print(df_cleaned)

             date        time                              aircraft  \
31     1919-03-23     unknown                      unnamed aircraft   
32     1919-04-07     unknown                      unnamed aircraft   
45     1919-05-19   12h 0m 0s                   farman f.60 goliath   
49     1919-05-31     unknown              blackburn r.t.1 kangaroo   
51     1919-06-15   8h 40m 0s         vickers fb.27 vimy commercial   
...           ...         ...                                   ...   
28529  2022-05-11     unknown         de havilland dhc-6 twin otter   
28530  2022-05-12    8h 4m 0s                           airbus a319   
28532  2022-05-23  18h 29m 0s  piper pa-61 aerostar (ted smith 601)   
28533  2022-05-24  15h 40m 0s              de havilland dhc-3 otter   
28534  2022-05-29   10h 7m 0s         de havilland dhc-6 twin otter   

                       operator                   flight_phase  \
31               private french                         flight   
32             